In [4]:
from metrics import *
from bootstrap import *
from metric_classes import *

<function condition_number_xtx at 0x7fcb16313950>


In [5]:
str(Cp())

"Mallow's Cp"

In [6]:
c = Condition_Number()
#c.evaluate(model)

In [7]:
class EvaluateInfo():
    def __init__(self, model, metrics = [], comparisons = [], characteristics = []):
        self.model = model
        self.characteristics = characteristics # characteristic of the fitted model itself
        self.metrics = metrics # compares y and y_pred
        self.comparisons = comparisons #compares y, y_pred, y_pred_with_reduced_features
        
        all_names = [str(el) for el in metrics] +\
                     [str(el) for el in comparisons] +\
                     [str(el) for el in characteristics]
        
        self.names = dict(zip(all_names, [i for i in range(len(all_names))]))
        print(self.names)
        
    def _calcQuality(self):     
        model = self.model
        for (m, mask) in enumerate(self.masks):
            reduced_X_test = self.X_test[:, mask]
            model.fit(self.X_train.T[mask].T, self.y_train)
            for (i, metric) in enumerate(self.metrics):
                self.quality[i][m] = metric.evaluate(model, reduced_X_test, self.y_test)

            for (i, comp) in enumerate(self.comparisons):
                ind = i + len(self.metrics)
                self.quality[ind][m] = comp.evaluate(self.full, model, self.X_test,
                                                                 reduced_X_test, self.y_test)
            for (i, char) in enumerate(self.characteristics):
                ind = i + len(self.metrics) + len(self.comparisons)
                self.quality[ind][m] = char.evaluate(model)
        
    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        if masks is None:
            masks = np.ones((1, X.shape[1]), dtype=bool)
        masks = np.array(masks, dtype=bool)
        if len(masks.shape) == 1:
            masks = masks.reshape((1, len(masks.shape)))
        self.masks = masks
        self.len_sample = len_sample
        
        self.n_samples = n_samples

        self.X_train = X_train
        self.y_train = y_train

        self.X_test = X_test
        self.y_test = y_test
        
        self.full = deepcopy(self.model.fit(self.X_train, self.y_train))
        
        self.result = np.zeros((len(self.metrics) + len(self.comparisons) 
                                + len(self.characteristics), len(self.masks), self.n_samples))
        
        self.quality = np.zeros((len(self.metrics) + len(self.comparisons) 
                                + len(self.characteristics), len(self.masks)))
        self._calcQuality()
        
    
    
    def get_quality(self):
        return self.quality
    
    def get_result(self):
        return self.result
    
    def _by_name(self, name, array):
        if name in self.names:
            return array[self.names[name]]
        else:
            raise NameError('Invalid argument')
        
    def quality_by_name(self, name):
        return self._by_name(name, self.quality)
            
    def result_by_name(self, name):
        return self._by_name(name, self.result)
            
    

In [8]:
from copy import deepcopy
class EvaluateStaticInfo(EvaluateInfo):
    def __init__(self, model, metrics = [], comparisons = []):
        super(EvaluateStaticInfo, self).__init__(model, metrics, comparisons, [])
    
    def __eval(self):
        model = self.model
        
        sample_X, sample_y = self.boot.values()
        
        self.models = []
        for mask in self.masks:
            self.models += [deepcopy(model.fit(self.X_train[:, mask], self.y_train))]
        
        for (m, mask) in enumerate(self.masks):
            for it in range(self.n_samples):
                reduced_X_cur = (sample_X[it].T[mask]).T
                for (i, metric) in enumerate(self.metrics):
                    self.result[i][m][it] = metric.evaluate(self.models[m], reduced_X_cur, sample_y[it])
                for(i, comp) in enumerate(self.comparisons):
                    ind = i + len(self.metrics)
                    self.result[ind][m][it] = comp.evaluate(self.full, self.models[m], sample_X[it], 
                                                              reduced_X_cur, sample_y[it])

    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        super(EvaluateStaticInfo, self).fit(X_train, y_train, X_test, y_test, masks, n_samples, len_sample)
        if self.len_sample is None:
            self.len_sample = len(X_test)
        self.boot = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        self.__eval()
        

In [9]:
from copy import deepcopy
class EvaluateDynamicInfo(EvaluateInfo):
    def __init__(self, model, metrics = [], comparisons = [], characteristics = []):
        super(EvaluateDynamicInfo, self).__init__(model, metrics, comparisons, characteristics)
    
    def __eval(self):
        model = self.model
        
        sample_X, sample_y = self.boot.values()
            
        for (m, mask) in enumerate(self.masks):
            reduced_X_test = (self.X_test.T[mask]).T
            for it in range(self.n_samples):
                model.fit((sample_X[it].T[mask]).T, sample_y[it])
                for (i, metric) in enumerate(self.metrics):
                    self.result[i][m][it] = metric.evaluate(model, reduced_X_test, self.y_test)

                for (i, char) in enumerate(self.characteristics):
                    ind = i + len(self.metrics)
                    self.result[ind][m][it] = char.evaluate(model)

                for (i, comp) in enumerate(self.comparisons):
                    ind = i + len(self.metrics) + len(self.comparisons)
                    self.result[ind][m][it] = comp.evaluate(self.full, model, self.X_test,
                                                              reduced_X_test, self.y_test)
                

    def fit(self, X_train, y_train, X_test, y_test, masks = None, n_samples=20, len_sample=None):
        super(EvaluateDynamicInfo, self).fit(X_train, y_train, X_test, y_test, masks, n_samples, len_sample)
        if self.len_sample is None:
            self.len_sample = len(X_test)
        self.boot = Bootstrap(self.X_test, self.y_test, self.n_samples, self.len_sample)
        self.__eval()
    

In [10]:
from sklearn.linear_model import LinearRegression
mo = LinearRegression()

In [12]:
import scipy.stats as sps

XX = sps.norm.rvs(size=(10, 5))
X2 = sps.norm.rvs(size=(10, 5))
zz = sps.uniform.rvs(size=(5, 3))
yy = XX @ zz + sps.norm.rvs(scale=0.1, size=(10, 3))
y2 = X2 @ zz + sps.norm.rvs(scale=0.1, size=(10, 3))
maskk = np.ones(XX.shape[1])
maskk[2]=0
maskk[1] = 0
maskk[3] = 0
print(maskk)

metric1 = RSS()
metric2 = Adjusted_Determination_Coefficient()
metric3 = Determination_Coefficient()
metric4 = VIF()
metric5 = BIC()
char1 = Condition_Number()
comp1 = Cp()

In [13]:
info = EvaluateStaticInfo(mo, [metric1, metric2, metric3, metric4, metric5],
                           [comp1])#, [char1])

{'RSS': 0, 'Adj Det Coef': 1, 'Det Coef': 2, 'VIF': 3, "Mallow's Cp": 5, 'BIC': 4}


In [14]:
info.fit(XX, yy, X2, y2, masks=[np.ones(len(XX.T)), maskk])

In [15]:
info.get_result()

array([[[  9.94578307e-01,   7.60067303e-01,   1.20417341e+00,
           8.70208642e-01,   3.41759889e-01,   1.27030466e+00,
           8.46708174e-01,   8.86550095e-01,   1.27226444e+00,
           1.18620340e+00,   1.00593406e+00,   1.32994489e+00,
           8.13576730e-01,   1.75989637e-01,   3.84627919e-01,
           6.41014145e-01,   8.06766605e-01,   1.54801902e+00,
           1.03262097e+00,   7.00309550e-01],
        [  1.78024867e+01,   1.69327747e+01,   7.35680026e+00,
           3.61896011e+01,   1.92513519e+01,   1.53830408e+01,
           2.01809144e+01,   2.15802909e+01,   1.37495138e+01,
           1.60174158e+01,   2.53812486e+01,   1.39598040e+01,
           1.83767488e+01,   1.03223355e+01,   1.64003208e+01,
           1.90406395e+01,   1.72086034e+01,   1.76022007e+01,
           1.96959472e+01,   1.12772630e+01]],

       [[  9.55633384e-01,   9.86840403e-01,   9.10383988e-01,
           9.79202835e-01,   9.94167971e-01,   9.04342990e-01,
           9.73208565e-0

In [16]:
info.get_quality()

array([[  0.89214448,  18.30514359],
       [  0.97313096,   0.44869725],
       [  0.96545409,   0.29118218],
       [ 37.21755089,   1.8138854 ],
       [ 12.40506995,  22.91031378],
       [  1.        ,  14.51813801]])

In [17]:
info.result_by_name("Mallow's Cp")

array([[  1.        ,   1.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   1.        ,   1.        ,
          1.        ,   1.        ,   1.        ,   1.        ],
       [ 11.89953246,  16.27799386,   0.10941927,  35.58726923,
         50.33005088,   6.10972553,  17.83455718,  18.34187417,
          4.80711945,   7.50309385,  19.23152313,   4.49652821,
         16.58760373,  52.65308729,  36.63944444,  23.70393039,
         15.3303368 ,   5.37079099,  13.07374321,  10.10325465]])